In [38]:
import datetime as dt
import os
from pathlib import Path

import xarray as xr
import numpy as np

### Leer los ficheros del FSLE de AVISO

Se importan los ficheros del FSLE de aviso para ver sus características

In [15]:
# Initial separation: 0.02 deg
# Final separation: 0.6 deg
# Maximum integration window: 200 days
# Backward advection
# Time span: 1994-2020
# Temporal resolution: 1 day
# Spatial resolution: 0.04 deg
# Spatial coverage : global

date = dt.datetime(2023, 8, 15)
path_output = Path("/home/SHARED/SATLINK/fsle/")

fsle = sorted(os.listdir(path_output / "aviso"))
print(fsle[0])
print(fsle[-1])

2023_07_17_dataset-duacs-nrt-global-allsat-madt-fsle_1692957862262.nc
2023_08_26_dataset-duacs-nrt-global-allsat-madt-fsle_1694677542838.nc


In [81]:
fname = [p for p in fsle if str(p).startswith(f"{date:%Y_%m_%d}")][0]
ds = xr.open_dataset(path_output / "aviso" / fname)
ds

<xarray.Dataset>
Dimensions:    (time: 1, lon: 9000, lat: 3000)
Coordinates:
  * time       (time) datetime64[ns] 2023-08-15
  * lon        (lon) float64 180.0 180.0 180.1 180.1 ... 539.8 539.9 539.9 539.9
  * lat        (lat) float64 -59.98 -59.94 -59.9 -59.86 ... 59.9 59.94 59.98
Data variables:
    fsle_max   (time, lat, lon) float32 ...
    theta_max  (time, lat, lon) float32 ...
Attributes: (12/30)
    CDI:                        Climate Data Interface version 1.9.5 (http://...
    Conventions:                CF-1.6
    institution:                CNES, CLS, LOCEAN, CTOH
    _NCProperties:              version=1|netcdflibversion=4.4.1|hdf5libversi...
    cdm_data_type:              Grid
    standard_name_vocabulary:   http://cf-pcmdi.llnl.gov/documents/cf-standar...
    ...                         ...
    separation:                 Initial separation in degrees: 0.02; Maximal ...
    summary:                    This dataset contains Backward-in-time FSLE p...
    start_time:                 2023-08-21 00:00:00
    end_time:                   2023-08-27 00:00:00
    History:                    Translated to CF-1.0 Conventions by Netcdf-Ja...
    CDO:                        Climate Data Operators version 1.9.5 (http://...

In [103]:
print(path_output / "aviso" / fname)

/home/SHARED/SATLINK/fsle/aviso/2023_08_15_dataset-duacs-nrt-global-allsat-madt-fsle_1694093586079.nc


In [100]:
mask = ds["fsle_max"].isel(time=0).isnull().data.T
mask.sum()

7403270

In [93]:
ds1 = xr.open_dataset("2023_08_15_fsle.nc")

In [94]:
ds1["lambda1"].isnull().sum()

<xarray.DataArray 'lambda1' ()>
array(193)

In [101]:
ds1["lambda1"].data[mask] = np.nan

In [99]:
ds1["lambda1"].isnull().sum()

<xarray.DataArray 'lambda1' ()>
array(7403312)

In [88]:
mask

<xarray.DataArray 'fsle_max' (lon: 9000, lat: 3000)>
array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])
Coordinates:
  * lon      (lon) float64 180.0 180.0 180.1 180.1 ... 539.8 539.9 539.9 539.9
  * lat      (lat) float64 -59.98 -59.94 -59.9 -59.86 ... 59.86 59.9 59.94 59.98

In [17]:
max(ds.lat.values), min(ds.lat.values)

(59.98000000000001, -59.98)

In [18]:
lon = ((ds.lon.values + 180) % 360) - 180

In [19]:
min(lon), max(lon)

(-179.98000427246097, 179.97999572753906)

### Leer corrientes (uv) de CMEMS

Se ven las características de los ficheros de corrientes descargados de CMEMS. Se comparan con las corrientes habituales de Satlink

In [20]:
uv = sorted(os.listdir(path_output / "duacs"))
print(uv[0])
print(uv[-1])

nrt_global_allsat_phy_l4_20230601_20230607.nc
nrt_global_allsat_phy_l4_20230831_20230906.nc


In [21]:
ds = xr.open_dataset(path_output / "duacs" / uv[0])
ds

<xarray.Dataset>
Dimensions:    (time: 1, latitude: 720, nv: 2, longitude: 1440)
Coordinates:
  * time       (time) datetime64[ns] 2023-06-01
  * latitude   (latitude) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * longitude  (longitude) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * nv         (nv) int32 0 1
Data variables: (12/13)
    crs        int32 ...
    lat_bnds   (latitude, nv) float32 ...
    lon_bnds   (longitude, nv) float32 ...
    sla        (time, latitude, longitude) float64 ...
    err_sla    (time, latitude, longitude) float64 ...
    ugosa      (time, latitude, longitude) float64 ...
    ...         ...
    vgosa      (time, latitude, longitude) float64 ...
    err_vgosa  (time, latitude, longitude) float64 ...
    adt        (time, latitude, longitude) float64 ...
    ugos       (time, latitude, longitude) float64 ...
    vgos       (time, latitude, longitude) float64 ...
    flag_ice   (time, latitude, longitude) float64 ...
Attributes: (12/44)
    Conventions:                     CF-1.6
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Grid
    comment:                         Sea Surface Height measured by Altimetry...
    contact:                         servicedesk.cmems@mercator-ocean.eu
    creator_email:                   servicedesk.cmems@mercator-ocean.eu
    ...                              ...
    summary:                         SSALTO/DUACS Near-Real-Time Level-4 sea ...
    time_coverage_duration:          P1D
    time_coverage_end:               2023-06-01T12:00:00Z
    time_coverage_resolution:        P1D
    time_coverage_start:             2023-05-31T12:00:00Z
    title:                           NRT merged all satellites Global Ocean G...

In [22]:
uv_old = sorted(os.listdir(path_output / "uv"))
print(uv_old[0])
print(uv_old[-1])

2023-02-07.nc
2023-09-04.nc


In [23]:
ds = xr.open_dataset(path_output / "uv" / uv_old[0])
ds

<xarray.Dataset>
Dimensions:    (latitude: 1441, longitude: 4320)
Coordinates:
    depth      float32 ...
  * latitude   (latitude) float32 -60.0 -59.92 -59.83 ... 59.83 59.92 60.0
    time       datetime64[ns] ...
  * longitude  (longitude) float32 -180.0 -179.9 -179.8 ... 179.8 179.8 179.9
Data variables:
    uo         (latitude, longitude) float32 ...
    vo         (latitude, longitude) float32 ...
Attributes: (12/13)
    producer:          CMEMS - Global Monitoring and Forecasting Centre
    title:             daily mean fields from Global Ocean Physics Analysis a...
    area:              Global
    Conventions:       CF-1.8
    credit:            E.U. Copernicus Marine Service Information (CMEMS)
    contact:           https://marine.copernicus.eu/contact
    ...                ...
    source:            MOI GLO12
    licence:           http://marine.copernicus.eu/services-portfolio/service...
    institution:       Mercator Ocean International
    _CoordSysBuilder:  ucar.nc2.dataset.conv.CF1Convention
    comment:           
    history:           Data extracted from dataset http://localhost:8080/thre...

### Leer FSLE calculado por `map_of_fsle`

In [24]:
ds = xr.open_dataset("2023_08_07_fsle.nc")

In [25]:
ds.lon.values

array([-179.98, -179.94, -179.9 , ...,  179.9 ,  179.94,  179.98])

In [26]:
ds

<xarray.Dataset>
Dimensions:  (lon: 9000, lat: 3000)
Coordinates:
  * lon      (lon) float64 -180.0 -179.9 -179.9 -179.9 ... 179.9 179.9 180.0
  * lat      (lat) float64 -59.98 -59.94 -59.9 -59.86 ... 59.86 59.9 59.94 59.98
Data variables:
    theta1   (lon, lat) float64 ...
    theta2   (lon, lat) float64 ...
    lambda1  (lon, lat) float64 ...
    lambda2  (lon, lat) float64 ...
Attributes: (12/21)
    title:                  Map of FSLE
    advection_time:         20 days, 0:00:00
    configuration:          sds.ini
    diagnostic:             False
    final_separation:       0.6
    initial_separation:     0.2
    ...                     ...
    verbose:                True
    version:                False
    x_max:                  179.98
    x_min:                  -179.98
    y_max:                  59.98
    y_min:                  -59.98